# Imports

In [1]:
import numpy as np
import itertools
import time
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import pixiedust

Pixiedust database opened successfully


# Defining functions

## Sigmoid

Sigmoid $\sigma(x) = \frac{1}{1+e^{-x}}$ and its derrivatives.

Sigmoid with parameter $t$
$$ \sigma_t(x) = \frac{1}{1+e^{-tx}}$$

In [2]:
T = 1

In [3]:
def sig(x):
    return 1 / (1 + np.exp(-T*x))

def sig1(x):
    return T * sig(x) * (1 - sig(x))

def sig2(x):
    return T * (sig1(x) - 2*sig(x)*sig1(x))

def sig3(x):
    return T * (sig2(x) - 2 * (sig1(x)**2 + sig(x) * sig2(x)))

def sig_pr(x, k):
    if k==0:
        return sig(x)
    if k==1:
        return sig1(x)
    if k==2:
        return sig2(x)
    if k==3:
        return sig3(x)

## RHS

The right side of the equation:  $\Delta\Psi(x,y) = 6 x (x-1) (1-2 y) + 2 y (y-1) (1-2 y)$

In [4]:
K=1
def f(x, y):
    return -(np.cos(np.sqrt(x**2 + y**2)) / np.sqrt(x**2 + y**2)) + np.sin(np.sqrt(x**2 + y**2))

The analytic solution is given by: $\ \Psi_a(x,y) = x (1-x) y (1-y) (1-2 y)$

In [5]:
# def psi_a(x, y):
#     return y * (1-y) * np.sin(np.pi * x)

def psi_a(x, y):
    return -np.sin(np.sqrt(x**2 + y**2))

$$\frac{\partial}{\partial x} \Psi_a = (1-2x) y (1-y) (1-2 y)$$

In [6]:
def psi_a_dx(x,y):
    return -(x * np.cos(np.sqrt(x**2 + y**2))) / np.sqrt(x**2 + y**2)

$$\frac{\partial}{\partial y} \Psi_a = x (1-x) (1-6y + 6y^2) $$

In [7]:
def psi_a_dy(x,y):
    return -(y * np.cos(np.sqrt(x**2 + y**2))) / np.sqrt(x**2 + y**2)

## Neural Network

The output of neural network $N(x,y,\vec{p})$, where $\vec{p} = [w, u, v]$:
$$N = \sum_i^H v_i \sigma(z_i) \text{, where } z_i = w_{i0} x + w_{i1} y + u_i$$

In [8]:
def z(x, y, p):
    z_x = np.multiply(x, p[0][0])
    z_y = np.multiply(y, p[0][1])
    z_ = np.array([z_x, z_y, p[1]]).sum(axis=0)
    return z_

def N(x, y, p):
    return np.sum(np.multiply(sig(z(x,y,p)), p[2]))

$$\frac{\partial^k N}{\partial x_j^k} = \sum_{i=1}^H v_i w_{ij}^k \sigma^{(k)}$$

In [9]:
def dN_dxj_k(x, y, p, j, k):
    wj = p[0][j]
    v = p[2]
    z_ = z(x, y, p)
    
    return np.sum(v * (wj**k) * sig_pr(z_, k))

$$\frac{\partial N}{\partial w_j} = x_j v \sigma '$$

In [10]:
def dN_dwj(x, y, p, j):
    xj = x if j==0 else y
    v = p[2]
    z_ = z(x, y, p)
    return xj * v * sig1(z_)

$$ \frac{\partial}{\partial w_j} \frac{\partial N}{\partial x_k} = x_j v w_k \sigma'' + v_i \sigma' \quad\text{ if } j = k$$

$$ \frac{\partial}{\partial w_j} \frac{\partial N}{\partial x_k} = x_j v w_k \sigma'' \quad\text{ if } j \neq k$$

In [11]:
def d_dwj_dN_dxk(x, y, p, j, k):
    xj = x if j==0 else y
    wk = p[0][k]
    jk = 1 if j==k else 0
    v = p[2]
    z_ = z(x, y, p)
    return xj * v * wk * sig2(z_) + jk * v * sig1(z_)

$$ \frac{\partial}{\partial w_j} \frac{\partial^2 N}{\partial x_k^2} = x_j v w_k^2 \sigma^{(3)} + 2 v w_k \sigma'' \quad\text{ if } j = k $$

$$ \frac{\partial}{\partial w_j} \frac{\partial^2 N}{\partial x_k^2} = x_j v w_k^2 \sigma^{(3)} \quad\text{ if } j \neq k $$

In [12]:
def d_dwj_dN2_dxk2(x, y, p, j, k):
    xj = x if j==0 else y
    wk = p[0][k]
    jk = 1 if j==k else 0
    v = p[2]
    z_ = z(x, y, p)
    return xj * v * (wk**2) * sig3(z_) + jk * 2 * v * wk * sig2(z_)

$$ \frac{\partial}{\partial u} \frac{\partial^k}{\partial x_j^k} N = v w_j^k \sigma^{(k+1)} $$

In [13]:
def d_du_dkN(x, y, p, j, k):
    v = p[2]
    wj = p[0][j]
    z_ = z(x, y, p)
    return v * (wj**k) * sig_pr(z_, k+1)

$$ \frac{\partial}{\partial v} \frac{\partial^k}{\partial x_j^k} N = w_j^k \sigma^{(k)} $$

In [14]:
def d_dv_dkN(x, y, p, j, k):
    wj = p[0][j]
    z_ = z(x, y, p)
    return (wj**k) * sig_pr(z_, k)

## Cost function

$$E[\vec{p}] = \sum_{i \in \hat{D}} \left\{ \frac{\partial^2 N}{\partial x^2} + \frac{\partial^2 N}{\partial y^2} - f(x,y) \right\}^2 
           +  \sum_{i \in \partial \hat{D}} N^2$$

In [15]:
BC=1

In [16]:
def error_term1(x, y, p):
    return dN_dxj_k(x,y,p,0,2)  +  dN_dxj_k(x,y,p,1,2)  -  f(x,y)

In [17]:
def cost(points, boundary_points, p):
    et1 = []
    et2 = []
    for x, y in points:
        err_sq = error_term1(x, y, p)**2
        et1.append(err_sq)
    
    for x, y in boundary_points:
        err_sq = N(x,y,p)**2
        et2.append(err_sq)
        
    cost = sum(et1) + BC*sum(et2)
    
    return cost

In [18]:
# def relative_err(all_points, p, dOmega):
#     trial1 = np.sum(dOmega*np.array([np.linalg.norm(N(x,y,p) - psi_a(x,y))**2
#                                      for x, y in all_points]))

#     trial2 = np.sum([dOmega*((dN_dxj_k(x,y,p,0,1)-psi_a_dx(x,y))**2 +
#                              (dN_dxj_k(x,y,p,1,1)-psi_a_dy(x,y))**2)
#                      for x, y in all_points])

#     ana1 = np.sum(dOmega*np.array([np.linalg.norm(psi_a(x,y))**2
#                                          for x, y in all_points]))
#     ana2 = np.sum([dOmega*(psi_a_dx(x,y)**2 + psi_a_dy(x,y)**2)
#                    for x, y in all_points])
    
#     relative_err = (trial1 + trial2)**(1/2) / (ana1 + ana2)**(1/2)

#     return relative_err

# def relative_err(all_points, p, dOmega):
#     #this abs error
#     dx=0.3
#     bx=0.1
#     points, b_points = get_points(dx,bx)
#     all_points = points + b_points
#     abs_err = np.sum(np.array([np.linalg.norm(N(x,y,p) - psi_a(x,y))
#                                      for x, y in all_points])) / len(all_points)
#     return abs_err

def relative_err(p, dx=0.3):
    r = np.arange(np.pi, 2*np.pi+dx, dx)
    th = np.arange(-np.pi, np.pi+dx, dx)
    R, TH = np.meshgrid(r,th)

    x = np.ravel(R*np.cos(TH))
    y = np.ravel(R*np.sin(TH))
    
    all_points = list(zip(x,y))

#     dOmega = area / len(all_points)
    dOmega = 3 * np.pi**3 / len(all_points)

    trial1 = np.sum(dOmega*np.array([np.linalg.norm(N(x,y,p) - psi_a(x,y))**2
                                     for x, y in all_points]))

    trial2 = np.sum([dOmega*((dN_dxj_k(x,y,p,0,1)-psi_a_dx(x,y))**2 +
                             (dN_dxj_k(x,y,p,1,1)-psi_a_dy(x,y))**2)
                     for x, y in all_points])

    ana1 = np.sum(dOmega*np.array([np.linalg.norm(psi_a(x,y))**2
                                         for x, y in all_points]))
    ana2 = np.sum([dOmega*(psi_a_dx(x,y)**2 + psi_a_dy(x,y)**2)
                   for x, y in all_points])
    
    rel_err = (trial1 + trial2)**(1/2) / (ana1 + ana2)**(1/2)
    
    return rel_err

# Gradients

$$ \frac{\partial E[\vec{p}]}{\partial w_j} = \sum_{i \in \hat{D}} \left\{ 2 \text{ (error_term1) } \left( \frac{\partial}{\partial w_j} \frac{\partial^2 N}{\partial x^2} + \frac{\partial}{\partial w_j} \frac{\partial^2 N}{\partial y^2} \right) \right\}  +  \sum_{i \in \partial \hat{D}} 2 N \frac{\partial N}{\partial w_j}$$

In [19]:
def dE_dwj(points, boundary_points, p, j):
    t1 = []
    t2 = []
    for x, y in points:
        tmp = 2 * error_term1(x,y,p) * (d_dwj_dN2_dxk2(x,y,p,j,0) + d_dwj_dN2_dxk2(x,y,p,j,1))
        t1.append(tmp)
        
    for x, y in boundary_points:
        tmp = 2 * N(x,y,p) * dN_dwj(x,y,p,j)
        t2.append(tmp)
    
    grad = sum(t1) + BC*sum(t2)
    
    return grad

$$ \frac{\partial E[\vec{p}]}{\partial u} = \sum_{i \in \hat{D}} \left\{ 2 \text{ (error_term1) } \left( \frac{\partial}{\partial u} \frac{\partial^2 N}{\partial x^2} + \frac{\partial}{\partial u} \frac{\partial^2 N}{\partial y^2} \right) \right\} +  \sum_{i \in \partial \hat{D}} 2 N \frac{\partial N}{\partial u}$$

In [20]:
def dE_du(points, boundary_points, p):
    t1 = []
    t2 = []
    for x, y in points:
        tmp = 2 * error_term1(x,y,p) * (d_du_dkN(x,y,p,0,2) + d_du_dkN(x,y,p,1,2))
        t1.append(tmp)
        
    for x, y in boundary_points:
        tmp = 2 * N(x,y,p) * d_du_dkN(x,y,p,0,0)
        t2.append(tmp)
    
    grad = sum(t1) + BC*sum(t2)
    
    return grad

$$ \frac{\partial E[\vec{p}]}{\partial v} = \sum_{i \in \hat{D}} \left\{ 2 \text{ (error_term1) } \left( \frac{\partial}{\partial v} \frac{\partial^2 N}{\partial x^2} + \frac{\partial}{\partial v} \frac{\partial^2 N}{\partial y^2} \right) \right\}  +  \sum_{i \in \partial \hat{D}} 2 N \frac{\partial N}{\partial v}$$

In [21]:
def dE_dv(points, boundary_points, p):
    t1 = []
    t2 = []
    for x, y in points:
        tmp = 2 * error_term1(x,y,p) * (d_dv_dkN(x,y,p,0,2) + d_dv_dkN(x,y,p,1,2))
        t1.append(tmp)
        
    for x, y in boundary_points:
        tmp = 2 * N(x,y,p) * d_dv_dkN(x,y,p,0,0)
        t2.append(tmp)
    
    grad = sum(t1) + BC*sum(t2)
    
    return grad

# NN class

In [22]:
def get_points(dx,bx):
    r = np.arange(np.pi, 2*np.pi+dx, dx)
    th = np.arange(-np.pi, np.pi+dx, dx)
    R, TH = np.meshgrid(r,th)

    r_bound = np.array([np.pi, 2*np.pi])
    th_bound = np.arange(-np.pi, np.pi+bx, bx)
    R_bound, TH_bound = np.meshgrid(r_bound,th_bound)

    x = np.ravel(R*np.cos(TH))
    y = np.ravel(R*np.sin(TH))

    x_bound = np.ravel(R_bound*np.cos(TH_bound))
    y_bound = np.ravel(R_bound*np.sin(TH_bound))

    boundary_points = list(zip(x_bound, y_bound))
    points = [(a,b) for a, b in zip(x,y) if (a,b) not in boundary_points]

    return points, boundary_points

In [23]:
class NNTrain:
    def __init__(self, dx=0.1, bx=0.08, hidden_nodes=10, alpha=0.01, 
                 beta_1=0.9, beta_2=0.999, epsilon=1e-8, update_interval=50,
                 if_rel_err=False, output_file='output/output.csv'):
        
        self.output_file = output_file
        self.training_started = False
        self.hidden_nodes = hidden_nodes
        self.alpha = alpha
        self.beta_1 = beta_1
        self.beta_2 = beta_2
        self.epsilon = epsilon
        self.update_interval = update_interval
        self.points, self.boundary_points = get_points(dx,bx)
        self.cost_rate = []
        self.if_rel_err = if_rel_err
        if self.if_rel_err:
            self.rel_err = []
        self.p = np.array([np.random.randn(2,hidden_nodes),
                           np.random.randn(hidden_nodes),
                           np.random.randn(hidden_nodes)])
        self.m_t = np.array([np.zeros(hidden_nodes),
                             np.zeros(hidden_nodes),
                             np.zeros(hidden_nodes),
                             np.zeros(hidden_nodes)])
        self.v_t = np.array([np.zeros(hidden_nodes),
                             np.zeros(hidden_nodes),
                             np.zeros(hidden_nodes),
                             np.zeros(hidden_nodes)])
        self.if_continue = np.array([True, True, True, True])
        
        
    def adam(self, t, w, g_t, theta_0):
        self.m_t[w] = self.beta_1 * self.m_t[w] + (1-self.beta_1) * g_t
        self.v_t[w] = self.beta_2 * self.v_t[w] + (1-self.beta_2) * (g_t * g_t)
        
        m_cap = self.m_t[w]/(1-(self.beta_1**t))
        v_cap = self.v_t[w]/(1-(self.beta_2**t))
        theta_0_prev = np.copy(theta_0)
        theta_0 = theta_0 - (self.alpha*m_cap)/(np.sqrt(v_cap)+self.epsilon)
        
        if(np.all(theta_0 == theta_0_prev)):
            self.if_continue[w] = False
            
        return theta_0
        

    def train(self, itr=1000):
        if self.training_started:
            mode='a'
        else:
            mode='w'
        self.training_started=True
        
        with open(self.output_file, mode) as file:
            if mode=='w':
                if self.if_rel_err:
                    file.write('Iteration,Cost,Cost Diff,Relative Error,Relative Error Diff\n')
                else:
                    file.write('Iteration,Cost,Cost Diff\n')
            start=len(self.cost_rate)-1
            if start<1:
                start+=1
                self.cost_rate.append(cost(self.points,self.boundary_points,self.p))
                if self.if_rel_err:
                    self.rel_err.append(relative_err(self.p, 0.4))

            i = start
            while i < start+itr:
                if(not np.any(self.if_continue)):
                    break
                                    
                #adam optimizer
                g_w0 = dE_dwj(self.points, self.boundary_points, self.p, 0)
                g_w1 = dE_dwj(self.points, self.boundary_points, self.p, 1)
                g_u = dE_du(self.points, self.boundary_points, self.p)
                g_v = dE_dv(self.points, self.boundary_points, self.p)
                
                if self.if_continue[0] :self.p[0][0] = self.adam(i+1, 0, g_w0, self.p[0][0])
                if self.if_continue[1] :self.p[0][1] = self.adam(i+1, 1, g_w1, self.p[0][1])
                if self.if_continue[2] :self.p[1] = self.adam(i+1, 2, g_u, self.p[1])
                if self.if_continue[3] :self.p[2] = self.adam(i+1, 3, g_v, self.p[2])
                
                
                self.cost_rate.append(cost(self.points,self.boundary_points,self.p))
                cost_diff = self.cost_rate[i]-self.cost_rate[i+1]
                if self.if_rel_err:
                    self.rel_err.append(relative_err(self.p, 0.4))
                    rel_diff = self.rel_err[i]-self.rel_err[i+1]

                if self.if_rel_err:
                    file.write('{},{:.8f},{:.8f},{:.8f},{:.8f}\n'.format(
                        i+1,self.cost_rate[i+1], cost_diff,
                        self.rel_err[i+1], rel_diff))
                else:
                    file.write('{},{:.8f},{:.8f}\n'.format(
                        i+1,self.cost_rate[i+1],
                        cost_diff))

                if i%self.update_interval==0:
                    file.flush()
                    
                i+=1
                
                
    def save_result(self, output_name=''):
        timestr = time.strftime("%Y%m%d-%H%M")
        np.savez('output/'+ timestr + '_' + output_name +'_nn_params.npz', self.p)
        np.savez('output/'+ timestr + '_' + output_name +'_cost_rate.npz', self.cost_rate)
        if self.if_rel_err:
            np.savez('output/'+ timestr + '_' + output_name +'_rel_err.npz', self.rel_err)

# Graphs

In [24]:
%matplotlib notebook
%matplotlib notebook

## Hidden Nodes

In [25]:
def plot_nodes(a, save_name=None, save=True, zlim=True):
    p = a.p
    hmax = a.p[0][0].shape[0]
    col = int(np.sqrt(hmax)) if int(np.sqrt(hmax)) < 10 else 10
    row = np.ceil(hmax / col)
    
    d_x=0.1

    fig = plt.figure(figsize=(col*3, row*3), dpi=100)
    for h in range(1,hmax+1):
        ax = fig.add_subplot(row,col,h, projection='3d')
        x = y = np.arange(0, 1+d_x, d_x)
        X, Y = np.meshgrid(x, y)

        zs = np.array([p[1][h-1]*sig(z(x,y,p))[h-1] for x,y in zip(np.ravel(X), np.ravel(Y))])
        Z = zs.reshape(X.shape)

        ax.set_xlabel('x')
        ax.set_ylabel('y')
        if zlim:
            ax.set_zlim(0,1)

        ax.plot_surface(X, Y, Z, cmap=cm.coolwarm)
        ax.title.set_text('HN: ' + str(h) + ', v=' + "{:.4E}".format(p[1][h-1]))
        time.sleep(0.2)

    if save==True:
        if save_name is None: 
            plt.savefig('t'+str(T)+'_h'+str(hmax)+'_n'+str(a.nx)+'.png')
        else:
            plt.savefig(save_name)
    else:
        plt.show()

## Error vs Iterations

In [26]:
def plot_iteration(arr, min_it=0, max_it=None):
    plt.figure()
    plt.plot(np.arange(len(arr[min_it:max_it])), np.array(arr[min_it:max_it]))
    plt.show()

## Solution Accuracy on the domain

In [27]:
def accuracy(p):
    d_x=0.01
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    x = y = np.arange(0, 1.0+d_x, d_x)
    X, Y = np.meshgrid(x, y)

    zs = np.array([psi_a(x,y)-psi_t(x,y,p) for x,y in zip(np.ravel(X), np.ravel(Y))])
    Z = zs.reshape(X.shape)

    ax.plot_surface(X, Y, Z, cmap=cm.coolwarm_r)
    # ax.ticklabel_format(style='sci', axis='z', scilimits=(0,0), useOffset=True, useMathText=True)


    ax.set_xlabel('x')
    ax.set_ylabel('y')
#     ax.set_zlabel('Accuracy')


    plt.show()

In [28]:
def plot_fun(fun, title=None, return_ax=False, xlim=None, ylim=None, zlim=None, **kwargs):
    dx=0.1

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    r = np.arange(np.pi, 2*np.pi+dx, dx)
    th = np.arange(-np.pi, np.pi+dx, dx)
    R, TH = np.meshgrid(r,th)
    X = R*np.cos(TH)
    Y = R*np.sin(TH)

    zs = np.array([fun(x,y,**kwargs) for x,y in zip(np.ravel(X), np.ravel(Y))])
    Z = zs.reshape(X.shape)

    ax.plot_surface(X, Y, Z, cmap=cm.coolwarm_r)

    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.title.set_text(title)
    
    if xlim:
        ax.set_xlim(xlim[0], xlim[1])
    if ylim:
        ax.set_ylim(ylim[0], ylim[1])
    if zlim:
        ax.set_zlim(zlim[0], zlim[1])

    if return_ax:
        return ax
    else:
        plt.show()

# Training

Meta training

In [29]:
a = NNTrain(dx=0.1,bx=0.01, hidden_nodes=20, alpha=0.01, update_interval=10, if_rel_err=False, output_file='out/out4.csv')

In [ ]:
BC=1
a.alpha=0.01
a.train(100)

In [ ]:
thresh=1e-4
for i in range(10000):
    if a.alpha < 1e-6:
        break
    if np.mean([np.absolute(s-t) for s, t in zip(a.cost_rate[-21:], a.cost_rate[-20:])]) < thresh:
        BC=BC*10
        a.alpha=a.alpha/10
    a.train(100)

Etc

In [47]:
relative_err(a.p, 0.02)

0.04144254483360438

In [31]:
relative_err(a.p, 0.02)

0.05005535131656225

In [32]:
relative_err(a.p, 0.05)

0.018723834558612432

In [46]:
plot_fun(N, p=a.p, zlim=(0,1.7))

<IPython.core.display.Javascript object>

In [48]:
plot_fun(lambda x,y,p: psi_a(x,y) - N(x,y,p), p=a.p)

<IPython.core.display.Javascript object>

In [49]:
a.save_result('eq2_4')

In [30]:
a.p = np.load('output/20191223-1733_eq2_4_nn_params.npz')['arr_0']